# Trabajo Práctico Experimental 7


Este trabajo tiene como objetivo explorar el funcionamiento e implementación de redes neuronales usando backpropagation

## Importación de Librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_blobs, make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

## Creación del Conjunto de Datos


En este caso, se hace uso de la utilidad make_moons de SciKit-learn para crear un conjunto de datos de dos clases en forma de lunas.

In [ ]:
n_samples = 400
noise = 0.08
random_state = 42 


X, y = make_moons(
    n_samples=n_samples,
    noise=noise,
    random_state=random_state
)


In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X[y == 0, 0], X[y == 0, 1], c='blue', label='Clase 0', alpha=0.6, edgecolors='w')
plt.scatter(X[y == 1, 0], X[y == 1, 1], c='red', label='Clase 1', alpha=0.6, edgecolors='w')
plt.title("Datos de Clasificación Binaria con 2 Clusters por Clase")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.legend()
plt.grid(True)
plt.show()

## Implementación de la Red Neuronal con Backpropagation


A continuación se muestra la implementación de la red neuronal, esta hace uso de varias funciones de activación (tanh, sigmoide, relu) y tiene una función softmax de salida, y este implementa un standard scaler al recibir datos para en entrenamiento y al hacer la predicción.

In [ ]:

class NeuralNetwork:
    def __init__(self, hidden_layers=(3, 6), activation='relu', learning_rate=0.01, seed=None):
        
        self.hidden_layers = hidden_layers
        self.activation = activation
        self.learning_rate = learning_rate
        self.seed = seed
        self.weights = []
        self.biases = []
        self.scaler = StandardScaler()
        
        if seed is not None:
            np.random.seed(seed)
    
    def _initialize_parameters(self, input_dim, output_dim):
        layer_dims = [input_dim] + list(self.hidden_layers) + [output_dim]
        
        for i in range(len(layer_dims) - 1):
            if self.activation == 'relu':
                scale = np.sqrt(2. / layer_dims[i])
            else:
                scale = np.sqrt(1. / layer_dims[i])
                
            self.weights.append(np.random.randn(layer_dims[i], layer_dims[i+1]) * scale)
            self.biases.append(np.zeros((1, layer_dims[i+1])))
    
    # Funciones de activación
    def _activation_fn(self, z, derivative=False):
        if self.activation == 'sigmoid':
            if derivative:
                s = self._activation_fn(z)
                return s * (1 - s)
            return 1 / (1 + np.exp(-z))
        
        elif self.activation == 'tanh':
            if derivative:
                return 1 - np.tanh(z)**2
            return np.tanh(z)
        
        elif self.activation == 'relu':
            if derivative:
                return (z > 0).astype(float)
            return np.maximum(0, z)
    
    def _forward_propagation(self, X):
        activations = [X]
        z_values = []
        
        for i in range(len(self.weights)):
            z = np.dot(activations[-1], self.weights[i]) + self.biases[i]
            a = self._activation_fn(z) if i < len(self.weights) - 1 else self._softmax(z)
            
            z_values.append(z)
            activations.append(a)
        
        return activations, z_values
    
    def _softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)
    
    # Método para calcular la pérdida
    def _compute_loss(self, y, y_hat):
        m = y.shape[0]
        log_likelihood = -np.log(y_hat[range(m), y])
        loss = np.sum(log_likelihood) / m
        return loss
    
    def _backward_propagation(self, X, y, activations, z_values):
        m = X.shape[0]
        gradients = {}
        layers = len(self.weights)
        
        dz = activations[-1]
        dz[range(m), y] -= 1
        dz /= m
        
        for l in reversed(range(layers)):
            gradients[f'dW{l}'] = np.dot(activations[l].T, dz)
            gradients[f'db{l}'] = np.sum(dz, axis=0, keepdims=True)
            
            if l > 0:
                dz = np.dot(dz, self.weights[l].T) * self._activation_fn(z_values[l-1], derivative=True)
        
        # Update parameters
        for l in range(layers):
            self.weights[l] -= self.learning_rate * gradients[f'dW{l}']
            self.biases[l] -= self.learning_rate * gradients[f'db{l}']
    
    def fit(self, X, y, epochs=1000, verbose=100):
        X = self.scaler.fit_transform(X)
        y = y.astype(int)
        
        input_dim = X.shape[1]
        output_dim = len(np.unique(y))
        self._initialize_parameters(input_dim, output_dim)
        
        for epoch in range(1, epochs + 1):
            activations, z_values = self._forward_propagation(X)
            
            loss = self._compute_loss(y, activations[-1])
            
            self._backward_propagation(X, y, activations, z_values)
            
            if verbose and epoch % verbose == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")
    
    def predict(self, X):
        X = self.scaler.transform(X)
        activations, _ = self._forward_propagation(X)
        return np.argmax(activations[-1], axis=1)
    
    def predict_proba(self, X):
        X = self.scaler.transform(X)
        activations, _ = self._forward_propagation(X)
        return activations[-1]


## División del conjunto de datos de entrenamiento y de prueba

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Creación del modelo y entrenamiento

In [ ]:
nn = NeuralNetwork(hidden_layers=(24, 12, 6), activation='tanh' , learning_rate=0.01, seed=42)
nn.fit(X_train, y_train, epochs=2000, verbose=100)

In [ ]:
predicciones = nn.predict(X_test)

## Evaluación del modelo

In [ ]:
cm = ConfusionMatrixDisplay(confusion_matrix(y_test, predicciones), display_labels=np.unique(y))
cm.plot(cmap=plt.cm.Blues)

In [ ]:
classification_rep = classification_report(y_test, predicciones)
print(classification_rep)

In [ ]:

accuracy = accuracy_score(y_test, predicciones)
print(f"\nTest Accuracy: {accuracy:.4f}")

### Ejemplo de un dato cerca de la frontera de decisión


Una de las ventajas de usar una capa softmax de salida, es que se puede saber las probabilidades de la pertenecia de un dato a las clases existentes, en este caso se muestra la probabilidad de pertenencia de un dato cerca de la frontera de decisión

In [ ]:
ex = nn.predict_proba(np.array([[-0.1, 0.5]]))
print(f"\nProbabilidades de clase para el ejemplo [-0.1, 0.5]: {ex}")

Se puede observar que una clase no es ampliamente dominante, lo cual nos puede indicar incertidumbre en la clasificación del dato.

## Resultados de la clasififcación de los datos originales

In [ ]:
preds = nn.predict(X)
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=preds, palette="Set1", marker=".")
plt.title("Resultados de la clasificación con la Red Neuronal")